In [116]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [117]:
raw_data = pd.read_csv('/Users/reyanshariff/Desktop/CompSci_Projects/Stroke_Prediction_Model/dataset/healthcare-dataset-stroke-data.csv')
data = raw_data.copy(deep=True)
data.interpolate(method='linear', inplace = True)
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,34.55,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,29.30,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.00,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.60,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.60,formerly smoked,0


In [122]:
from sklearn.preprocessing import OneHotEncoder 
   
data['gender'] = data['gender'].astype('category') 
data['ever_married'] = data['ever_married'].astype('category') 
data['work_type'] = data['work_type'].astype('category') 
data['Residence_type'] = data['Residence_type'].astype('category') 

data['Gen_new'] = data['gender'].cat.codes 
data['evm_new'] = data['ever_married'].cat.codes 
data['wktp_new'] = data['work_type'].cat.codes
data['restype_new'] = data['Residence_type'].cat.codes  
  
# Create an instance of One-hot-encoder 
enc = OneHotEncoder() 
  
# Passing encoded columns 
  
enc_data = pd.DataFrame(enc.fit_transform( 
    data[['Gen_new', 'evm_new', 'wktp_new', 'restype_new']]).toarray()) 
  
# Merge with main 
data = pd.concat([data.drop(data.columns[[1, 5, 6, 7]], axis=1), enc_data], axis=1)
data

KeyError: 'gender'

In [119]:
smokestatus_unknown = data[data['smoking_status'] == 'Unknown']
df1 = pd.DataFrame(smokestatus_unknown)
smokestatus_known =  data[data['smoking_status'] != 'Unknown']
df2 = pd.DataFrame(smokestatus_known)
smokestatus_unknown.columns = smokestatus_unknown.columns.astype(str)
smokestatus_known.columns = smokestatus_known.columns.astype(str)

X = smokestatus_known.drop('smoking_status', axis=1)
Y = smokestatus_known['smoking_status']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

forest = RandomForestClassifier(n_estimators=45, max_depth=25, random_state=False, 
                                max_features=0.6, min_samples_leaf=3, n_jobs=-1)


forest.fit(X_train, Y_train)
y_pred_train = forest.predict(X_train)
y_pred = forest.predict(X_test)

In [120]:
smokestatus_unknown = smokestatus_unknown.drop('smoking_status', axis=1, errors='ignore')
smokestatus_unknown.columns = smokestatus_unknown.columns.astype(str)
prediction = forest.predict(smokestatus_unknown)
df1['smoking_status'] = prediction
data_combined = pd.concat([df1, df2])
data_combined

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke,Gen_new,evm_new,...,2,3,4,5,6,7,8,9,10,11
8,27419,59.0,0,0,76.15,23.50,never smoked,1,0,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,60491,78.0,0,0,58.57,24.20,never smoked,1,0,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
13,8213,78.0,0,1,219.84,27.75,formerly smoked,1,1,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
19,25226,57.0,0,1,217.08,30.10,formerly smoked,1,1,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
23,64778,82.0,0,1,208.30,32.50,formerly smoked,1,1,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,57.0,0,0,77.93,21.70,never smoked,0,0,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5105,18234,80.0,1,0,83.75,29.30,never smoked,0,0,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5106,44873,81.0,0,0,125.20,40.00,never smoked,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5107,19723,35.0,0,0,82.99,30.60,never smoked,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [121]:
drop_these = ['id', 'smoking_status']
X = data_combined.drop(drop_these, axis=1)
X.columns = X.columns.astype(str)
y = data_combined['smoking_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.60
